In [29]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [30]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 8 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_brand_listing             38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
dtypes: float64(7), int64(1)
memory usage: 2.7+ MB


In [31]:
df2 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
#df3 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
#df8 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2).fillna(0)
#df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
#df_unidos = df_unidos.join(df8).fillna(0)
df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10).fillna(0)
#df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)

df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 34 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_brand_listing             38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
mas_prom                       38829 non-null float64
vio_mas_apple                  38829 non-null float64
vio_mas_samsung                38829 non-null float64
vio_mas_motorola               38829 non-null float64
compro_apple                   38829 non-null float64
check_apple                    38829 non-null float64
check_samsung                  38829 non-null float64
timestamp                      38829 non-null int64
min_timestamp             

In [32]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_test = df_unidos.join(labels, how= "inner")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, ad93850f to d8d6b0da
Data columns (total 35 columns):
cant_conversions               19414 non-null float64
cant_checkouts                 19414 non-null float64
cant_viewed_product            19414 non-null float64
cant_searched_product          19414 non-null float64
cant_brand_listing             19414 non-null float64
total_sesiones                 19414 non-null int64
promedio_eventos_por_sesion    19414 non-null float64
proprosion                     19414 non-null float64
mas_prom                       19414 non-null float64
vio_mas_apple                  19414 non-null float64
vio_mas_samsung                19414 non-null float64
vio_mas_motorola               19414 non-null float64
compro_apple                   19414 non-null float64
check_apple                    19414 non-null float64
check_samsung                  19414 non-null float64
timestamp                      19414 non-null int64
min_timestamp             

In [33]:
features = df_test.columns.tolist()
features.remove('label')

In [34]:
rf = RandomForestClassifier(n_estimators=10000, n_jobs=-1, min_samples_split=150,\
                            class_weight='balanced')

#  Partimos los datos que tenemos para entrenar en dos partes, una para entrenar el modelo (80% de los datos)
#  y la otra parte se usará para probar el modelo (el 20% restante)
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20)

In [36]:
print("Train a Random Forest model")
rf.fit(X_train,Y_train)
rf.score(X_test,Y_test)

0.81457635848570698

In [41]:
"""X, y = df_test.iloc[:,:-1],df_test.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)"""

'X, y = df_test.iloc[:,:-1],df_test.iloc[:,-1]\ndata_dmatrix = xgb.DMatrix(data=X,label=y)\nX_train, X_test, y_train, y_test =     train_test_split(X, y, test_size=0.2, random_state=123)'

In [42]:
print("Train a XGBoost model")
xg_reg = XGBClassifier(max_depth=1,learning_rate=0.01,n_estimators=200,scale_pos_weight = 2,silent=True,n_jobs=-1)
xg_reg.fit(X_train,y_train)

Train a XGBoost model


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=1, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=2,
       seed=None, silent=True, subsample=1)

In [43]:
best_probs = (rf.predict_proba(X_test) + xg_reg.predict(xgb.DMatrix(X_test)))/2

best_proba

TypeError: can not initialize DMatrix from DMatrix